## Folium 이용해서 지도에 서울시 초등학교 표시해보기

데이터 셋 제공 : https://data.seoul.go.kr/dataList/datasetView.do?infId=OA-12059&srvType=S&serviceKind=1&currentPageNo=1

In [2]:
import pandas as pd
import folium as fl
import numpy as np
import requests

한글 인코딩 에러날때 [참고](https://soooprmx.com/archives/4912)

In [9]:
dataset = pd.read_csv('data/elem_school_seoul.csv', encoding='CP949', header = 1)

In [10]:
dataset.head()

,서대문구,서울안산초등학교,"서울 서대문구 통일로 321 (홍제동,서울안산초등학교)"
0,은평구,서울역촌초등학교,"서울 은평구 은평로3길 25 , 서울역촌초등학교 (역촌동,서울역촌초등학교)"
1,서대문구,서울연가초등학교,"서울 서대문구 증가로12가길 49-50 (남가좌동,연가초등학교)"
2,은평구,서울연광초등학교,"서울특별시 은평구 연서로35길 37 (불광동,연광초등학교)"
3,은평구,서울연신초등학교,"서울 은평구 연서로 347 (불광동,연신초등학교)"
4,은평구,서울연은초등학교,서울특별시 은평구 백련산로2길 35 (응암동)


In [23]:
dataset.columns = [ 'region','name', 'address']

In [24]:
dataset.head()

,region,name,address
0,은평구,서울역촌초등학교,"서울 은평구 은평로3길 25 , 서울역촌초등학교 (역촌동,서울역촌초등학교)"
1,서대문구,서울연가초등학교,"서울 서대문구 증가로12가길 49-50 (남가좌동,연가초등학교)"
2,은평구,서울연광초등학교,"서울특별시 은평구 연서로35길 37 (불광동,연광초등학교)"
3,은평구,서울연신초등학교,"서울 은평구 연서로 347 (불광동,연신초등학교)"
4,은평구,서울연은초등학교,서울특별시 은평구 백련산로2길 35 (응암동)


In [25]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 3 columns):
region     598 non-null object
name       598 non-null object
address    598 non-null object
dtypes: object(3)
memory usage: 14.1+ KB


도로명 주소를 기반으로 좌표값 얻어오기
https://www.vworld.kr/v4po_svrint_a002.do 참고

API를 이용하려면 API KEY 발급받아야함

In [27]:
coordinates = []
for data in dataset.iterrows():
    res = requests.get('http://apis.vworld.kr/new2coord.do?q=' + data[1]['address'] + '&apiKey=발급받은 API KEY&domain=http://map.vworld.kr/&output=json').json()
    x = res.get('EPSG_4326_X')
    y = res.get('EPSG_4326_Y')
    if x is not None and y is not None:
        coordinate = [float(y),float(x)]  # folium에 이용하기 위해 실수값으로 변환
        coordinates.append(coordinate)
    else:
        coordinates.append([0.0, 0.0])
   

In [28]:
len(coordinates)

598

In [33]:
dataset['coordinate'] = coordinates

In [34]:
dataset.head()

,region,name,address,coordinate
0,은평구,서울역촌초등학교,"서울 은평구 은평로3길 25 , 서울역촌초등학교 (역촌동,서울역촌초등학교)","[37.601586928, 126.91392844]"
1,서대문구,서울연가초등학교,"서울 서대문구 증가로12가길 49-50 (남가좌동,연가초등학교)","[37.580080643, 126.917814891]"
2,은평구,서울연광초등학교,"서울특별시 은평구 연서로35길 37 (불광동,연광초등학교)","[37.623856092, 126.92410867]"
3,은평구,서울연신초등학교,"서울 은평구 연서로 347 (불광동,연신초등학교)","[37.626825047, 126.92829666]"
4,은평구,서울연은초등학교,서울특별시 은평구 백련산로2길 35 (응암동),"[37.590494222, 126.925047544]"


### 지도그리기  - Folium   

Folium은 지도데이터에 'Leaflet.js'를 이용하여 위치정보를 시각화하기위한 라이브러리.  
기본적으로 'GeoJSON' 형식 또는 'topoJSON' 형식으로 데이터를 지정하면, 오버레이를 통해 마커의 형태로 위치정보를 지도상에 표현가능.  
  
[folium 공식문서](http://python-visualization.github.io/folium/docs-v0.5.0/)   
[참고](https://snscrawler.wordpress.com/tag/folium/)   

In [36]:
pivot_loc = [37.566345, 126.977893] # 중심좌표값 설정, '서울시청'
map_osm = fl.Map(location = pivot_loc, zoom_start = 8)
for data in dataset.iterrows():
    address = data[1]['address']
    coordinate = data[1]['coordinate']
    fl.Marker(coordinate, popup=address).add_to(map_osm)
    map_osm.save('data/elem_school_map_seoul.html')
    

![content01](/assets/elem_school_seoul.PNG)  
   